In [1]:
from pathlib import Path

from ray import tune
from ray.air import Checkpoint, session

In [2]:
ray_results_path = Path("~/ray_results").expanduser()
assert ray_results_path.is_dir()

In [3]:
last_experiment = max(ray_results_path.glob('TrainLightGCN_*'), key=lambda x: x.stat().st_ctime)
print(f"Loading results from {last_experiment}...")
assert last_experiment.is_dir()

class TrainLightGCN:
    def __init__(self):
        raise NotImplementedError

# TODO: Create an experiment grid or whatever is called
tuner = tune.Tuner.restore(str(last_experiment), TrainLightGCN)
rg = tuner.get_results()
rg.get_best_result()

Loading results from /home/daviddavo/ray_results/TrainLightGCN_2023-10-25_09-51-27...


2023-10-25 15:40:17,197	WARNING experiment_analysis.py:205 -- Failed to fetch metrics for 71 trial(s):
- TrainLightGCN_b989458f: FileNotFoundError('Could not fetch metrics for TrainLightGCN_b989458f: both result.json and progress.csv were not found at /home/daviddavo/ray_results/TrainLightGCN_2023-10-25_09-51-27/TrainLightGCN_b989458f_271_trial_index=2,batch_size=4,conv_layers=4,embedding_dim=370,l2=0.0000,learning_rate=0.0015_2023-10-25_15-39-27')
- TrainLightGCN_4d6d0341: FileNotFoundError('Could not fetch metrics for TrainLightGCN_4d6d0341: both result.json and progress.csv were not found at /home/daviddavo/ray_results/TrainLightGCN_2023-10-25_09-51-27/TrainLightGCN_5aec972a_324_trial_index=3,batch_size=4,conv_layers=4,embedding_dim=410,l2=0.0000,learning_rate=0.0122_2023-10-25_15-38-14')
- TrainLightGCN_245c76f4: FileNotFoundError('Could not fetch metrics for TrainLightGCN_245c76f4: both result.json and progress.csv were not found at /home/daviddavo/ray_results/TrainLightGCN_2023-1

Result(
  metrics={'iteration': 50, 'loss': 0.0015201962733498368, 'mf_loss': 0.0014900981229139573, 'emb_loss': 3.009814696535508e-05, 'recall': 0.00013293077109681817, 'ndcg': 0.0009264420200012466, 'precision': 0.0009913258983890955, 'map': 5.211509083875874e-05},
  path='/home/daviddavo/ray_results/TrainLightGCN_2023-10-25_09-51-27/TrainLightGCN_0f15f5f5_101_trial_index=0,batch_size=4,conv_layers=2,embedding_dim=440,l2=0.0000,learning_rate=0.0009_2023-10-25_10-18-45',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/home/daviddavo/ray_results/TrainLightGCN_2023-10-25_09-51-27/TrainLightGCN_0f15f5f5_101_trial_index=0,batch_size=4,conv_layers=2,embedding_dim=440,l2=0.0000,learning_rate=0.0009_2023-10-25_10-18-45/checkpoint_000000)
)

In [4]:
# Read dataframe and drop some unneeded columns
rdf = rg.get_dataframe('precision', 'max').drop(columns=['timestamp', 'node_ip', 'pid', 'hostname', 'time_since_restore', 'iterations_since_restore', 'logdir'])
rdf = rdf.drop(columns=['date'])
# rdf = rdf[rdf['done']]
print(rdf.columns)

for c in ['config/batch_size']: #, 'config/embedding_dim']:
    rdf[c] = 2**rdf[c]

rdf[rdf['config/__trial_index__'] >= 3].sort_values('precision').tail()

Index(['iteration', 'loss', 'mf_loss', 'emb_loss', 'recall', 'ndcg',
       'precision', 'map', 'done', 'training_iteration', 'trial_id',
       'time_this_iter_s', 'time_total_s', 'checkpoint_dir_name',
       'config/batch_size', 'config/embedding_dim', 'config/conv_layers',
       'config/learning_rate', 'config/l2', 'config/__trial_index__'],
      dtype='object')


,iteration,loss,mf_loss,emb_loss,recall,ndcg,precision,map,done,training_iteration,trial_id,time_this_iter_s,time_total_s,checkpoint_dir_name,config/batch_size,config/embedding_dim,config/conv_layers,config/learning_rate,config/l2,config/__trial_index__
178,35,182.447178,182.435523,1.165504e-02,0.002343,0.007860,0.006955,0.001792,False,7,4cc6c008,231.779568,1946.942651,None,32,100,2,0.90220,3.059893e-09,3
148,25,0.011212,0.011209,3.572330e-06,0.001732,0.007344,0.006955,0.001184,False,5,56138d88,15.266137,78.935414,None,256,155,3,0.00246,6.066749e-08,3
123,5,0.021675,0.012864,8.811053e-03,0.001697,0.007549,0.007464,0.001048,False,1,3cc2e0be,18.471001,18.471001,None,256,205,2,0.00946,2.195997e-04,3
152,35,0.057645,0.015744,4.190048e-02,0.002787,0.008889,0.007634,0.001823,False,7,a81850d3,28.706823,202.818440,None,128,110,2,0.02281,6.857073e-04,3
254,25,0.008283,0.008282,1.151678e-07,0.001738,0.008229,0.007634,0.001193,False,5,c1a64a1a,24.780501,124.924080,None,256,175,4,0.00472,1.045362e-09,3
